In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add the parent directory to the system path to import simulator
sys.path.insert(0, os.path.abspath('..'))
from simulator import run_simulation, get_sp500_residuals, create_ar_model

# --- SHARED SETTINGS ---
initial_net_worth = 1_000_000
annual_spend = 40_000 # 4% initial withdrawal rate
years = 30
panic_threshold = -0.15
inflation_rate = 0.03
n_paths = 1000
history_years = 60

# Setup Market Data (Using Historical Residuals for consistency)
print("Fetching Market Data...")
mu, residuals, _ = get_sp500_residuals(history_years)

# ==============================================================================
# SCENARIO A: SMART STRATEGY (Buffer + Flex Spending)
# ==============================================================================
print("\nRunning Scenario A: Smart Hedged Strategy...")
print("- 2 Year Cash Buffer")
print("- 4% Spending Cap (Reduces spending if portfolio drops)")

res_A = run_simulation(
    initial_net_worth=initial_net_worth,
    annual_spend=annual_spend,
    buffer_years=2,          # SAFETY 1: Cash Buffer
    years=years,
    panic_threshold=panic_threshold,
    inflation_rate=inflation_rate,
    n_paths=n_paths,
    mu=mu,
    residuals=residuals,
    use_ar_model=False,
    spending_cap_pct=0.04    # SAFETY 2: Dynamic Spending
)
port_A = res_A['portfolio_values']
withdrawals_A = res_A['withdrawal_values']

# ==============================================================================
# SCENARIO B: RECKLESS STRATEGY (All Equity + Fixed Spending)
# ==============================================================================
print("\nRunning Scenario B: Reckless Strategy...")
print("- 0 Year Cash Buffer (100% Equity)")
print("- Fixed Real Spending (No reduction, ignores portfolio value)")

res_B = run_simulation(
    initial_net_worth=initial_net_worth,
    annual_spend=annual_spend,
    buffer_years=0,          # RISKY: No Buffer
    years=years,
    panic_threshold=panic_threshold,
    inflation_rate=inflation_rate,
    n_paths=n_paths,
    mu=mu,
    residuals=residuals,
    use_ar_model=False,
    spending_cap_pct=1.0     # RISKY: Cap set to 100% effectively means "Withdraw whatever is needed"
)
port_B = res_B['portfolio_values']
withdrawals_B = res_B['withdrawal_values']

# ==============================================================================
# ANALYSIS: RUIN PROBABILITY
# ==============================================================================
ruin_A = np.mean(port_A[-1, :] <= 0)
ruin_B = np.mean(port_B[-1, :] <= 0)

print(f"\nResults:")
print(f"Scenario A Ruin Probability: {ruin_A:.1%}")
print(f"Scenario B Ruin Probability: {ruin_B:.1%}")

# ==============================================================================
# VISUALIZATION: RISK PATH COMPARISON (5th Percentile)
# ==============================================================================
# We want to find a specific "Bad Luck" path that is common to both (using same seeds/residuals would be ideal, 
# but simulation doesn't expose seed control easily per path. 
# Instead, we compare the 5th percentile outcome of each independent run).

# Find 5th percentile index for A
alpha = 0.05
final_A = port_A[-1, :]
rank_A = int(n_paths * alpha)
idx_A = np.argsort(final_A)[rank_A]

# Find 5th percentile index for B
final_B = port_B[-1, :]
rank_B = int(n_paths * alpha)
idx_B = np.argsort(final_B)[rank_B]

# Extract trajectories
traj_A = port_A[:, idx_A]
traj_B = port_B[:, idx_B]

# Extract withdrawals
wd_A = withdrawals_A[:, idx_A]
wd_B = withdrawals_B[:, idx_B]

# Plot 1: Portfolio Value Survival
plt.figure(figsize=(12, 6))
plt.plot(traj_A, label=f'Smart Strategy (Final: ${traj_A[-1]:,.0f})', color='green', linewidth=2)
plt.plot(traj_B, label=f'Reckless Strategy (Final: ${traj_B[-1]:,.0f})', color='red', linewidth=2, linestyle='--')
plt.axhline(0, color='black', linewidth=1)
plt.title(f"Survival Comparison: 5th Percentile Risk Scenario")
plt.ylabel("Portfolio Value ($)")
plt.xlabel("Years")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Plot 2: Income Protection vs Reduction
plt.figure(figsize=(12, 6))
plt.plot(wd_A, label='Smart Withdrawal (Dynamic)', color='green', linewidth=2)
plt.plot(wd_B, label='Reckless Withdrawal (Fixed)', color='red', linewidth=2, linestyle='--')
plt.axhline(annual_spend, color='gray', linestyle=':', label='Initial Target Spend (Real)')
plt.title("Income Stream in Risk Scenario (Real $)")
plt.ylabel("Annual Withdrawal ($)")
plt.xlabel("Years")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nObservation:")
print("The 'Smart' strategy survives by reducing spending (Green line drops) during bad market sequences.")
print("The 'Reckless' strategy maintains spending (Red line flat) until the portfolio hits $0, resulting in total ruin.")


2025-11-30 14:25:14.219 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-30 14:25:14.220 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-30 14:25:14.270 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-30 14:25:14.271 No runtime found, using MemoryCacheStorageManager
2025-11-30 14:25:14.271 No runtime found, using MemoryCacheStorageManager


Fetching Market Data...

Running Scenario A: Smart Hedged Strategy...
- 2 Year Cash Buffer
- 4% Spending Cap (Reduces spending if portfolio drops)


TypeError: run_simulation() got an unexpected keyword argument 'mu'